In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a scikit-learn model with Vertex AI SDK 2.0 and Bigframes

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vertex_ai_sdk/remote_training_bigframes_sklearn.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vertex_ai_sdk/remote_training_bigframes_sklearn.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
    <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/vertex_ai_sdk/remote_training_bigframes_sklearn.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
</table>

## Overview

This tutorial demonstrates how to train a scikit-learn model using Vertex AI local-to-remote training with Vertex AI SDK 2.0 and BigQuery Bigframes as the data source.

Learn more about [bigframes](https://cloud.google.com/bigquery/docs/).

### Objective

In this tutorial, you learn to use `Vertex AI SDK 2.0` with Bigframes as input data source.


This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI Remote Training`


The steps performed include:

- Initialize a dataframe from a BigQuery table and split the dataset
- Perform transformations as a Vertex AI remote training.
- Train the model remotely and evaluate the model locally

**Local-to-remote training**

```
import vertexai
from my_module import MyModelClass

vertexai.preview.init(remote=True, project="my-project", location="my-location", staging_bucket="gs://my-bucket")

# Wrap the model class with `vertex_ai.preview.remote`
MyModelClass = vertexai.preview.remote(MyModelClass)

# Instantiate the class
model = MyModelClass(...)

# Optional set remote config
model.fit.vertex.remote_config.display_name = "MyModelClass-remote-training"
model.fit.vertex.remote_config.staging_bucket = "gs://my-bucket"

# This `fit` call will be executed remotely
model.fit(...)
```

### Dataset

This tutorial uses the <a href="https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html">IRIS dataset</a>, which predicts the iris species.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
[BigQuery pricing](https://cloud.google.com/bigquery/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 

In [ ]:
# Install the packages
! pip3 install --upgrade --quiet google-cloud-aiplatform[preview]
! pip3 install --upgrade --quiet bigframes

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries and define constants

In [ ]:
import bigframes.pandas as bf
import vertexai

bf.options.bigquery.location = "us"  # Dataset is in 'us' not 'us-central1'
bf.options.bigquery.project = PROJECT_ID

from bigframes.ml.model_selection import \
    train_test_split as bf_train_test_split

REMOTE_JOB_NAME = "sdk2-bigframes-sklearn"
REMOTE_JOB_BUCKET = f"{BUCKET_URI}/{REMOTE_JOB_NAME}"

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
)

## Prepare the dataset

Now load the Iris dataset and split the data into train and test sets.

In [ ]:
df = bf.read_gbq("bigquery-public-data.ml_datasets.iris")

species_categories = {
    "versicolor": 0,
    "virginica": 1,
    "setosa": 2,
}
df["species"] = df["species"].map(species_categories)

# Assign an index column name
index_col = "index"
df.index.name = index_col

In [ ]:
feature_columns = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
label_columns = df[["species"]]
train_X, test_X, train_y, test_y = bf_train_test_split(
    feature_columns, label_columns, test_size=0.2
)

print("X_train size: ", train_X.size)
print("X_test size: ", test_X.size)

## Feature transformation

Next, you do feature transformations on the data using the Vertex AI remote training service.

First, you re-initialize Vertex AI to enable remote training.

In [ ]:
# Switch to remote mode for training
vertexai.preview.init(remote=True)

### Execute remote job for fit_transform() on training data

Next, indicate that the `StandardScalar` class is to be executed remotely. Then set up the data transform and call the `fit_transform()` method is executed remotely.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Wrap classes to enable Vertex remote execution
StandardScaler = vertexai.preview.remote(StandardScaler)

# Instantiate transformer
transformer = StandardScaler()

# Set training config
transformer.fit_transform.vertex.remote_config.display_name = (
    f"{REMOTE_JOB_NAME}-fit-transformer-bigframes"
)
transformer.fit_transform.vertex.remote_config.staging_bucket = REMOTE_JOB_BUCKET

# Execute transformer on Vertex (train_X is bigframes.dataframe.DataFrame, X_train is np.array)
X_train = transformer.fit_transform(train_X)

### Remote transform on test data

In [ ]:
# Transform test dataset before calculate test score
transformer.transform.vertex.remote_config.display_name = (
    REMOTE_JOB_NAME + "-transformer"
)
transformer.transform.vertex.remote_config.staging_bucket = REMOTE_JOB_BUCKET

# Execute transformer on Vertex (test_X is bigframes.dataframe.DataFrame, X_test is np.array)
X_test = transformer.transform(test_X)

## Remote training

First, train the scikit-learn model as a remote training job:

- Set LogisticRegression for the remote training job.
- Invoke LogisticRegression locally which will launch the remote training job.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Wrap classes to enable Vertex remote execution
LogisticRegression = vertexai.preview.remote(LogisticRegression)

# Instantiate model, warm_start=True for uptraining
model = LogisticRegression(warm_start=True)

# Set training config
model.fit.vertex.remote_config.display_name = REMOTE_JOB_NAME + "-sklearn-model"
model.fit.vertex.remote_config.staging_bucket = REMOTE_JOB_BUCKET

# Train model on Vertex
model.fit(train_X, train_y)

## Remote prediction

Obtain predictions from the trained model.

In [ ]:
# Remote evaluation
vertexai.preview.init(remote=True)

# Evaluate model's accuracy score
predictions = model.predict(test_X)

print(f"Remote predictions: {predictions}")

## Local evaluation

Score model results locally.

In [ ]:
# User must convert bigframes to pandas dataframe for local evaluation
train_X_pd = train_X.to_pandas().reset_index(drop=True)
train_y_pd = train_y.to_pandas().reset_index(drop=True)

test_X_pd = test_X.to_pandas().reset_index(drop=True)
test_y_pd = test_y.to_pandas().reset_index(drop=True)

In [ ]:
# Switch to local mode for testing
vertexai.preview.init(remote=False)

# Evaluate model's accuracy score
print(f"Train accuracy: {model.score(train_X_pd, train_y_pd)}")

print(f"Test accuracy: {model.score(test_X_pd, test_y_pd)}")

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
import os

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI